In [3]:
# ------------------------------
# SCRAPER + SQLITE COMPLETO (CORREGIDO PRECIOS)
# ------------------------------

import time
import re
import sqlite3
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
import random

# ------------------------------
# CONFIGURACIÓN SCRAPER
# ------------------------------
URL_BASE = "https://books.toscrape.com/"
CABECERAS = {"User-Agent": "Mozilla/5.0 (educational scraping)"}
MAPA_RATING = {"Zero":0, "One":1, "Two":2, "Three":3, "Four":4, "Five":5}

# Sesión requests
sesion = requests.Session()
sesion.headers.update(CABECERAS)

# ------------------------------
# FUNCIONES SCRAPER
# ------------------------------
def obtener_sopa(url, reintentos=3, espera=1.0):
    for intento in range(reintentos):
        try:
            respuesta = sesion.get(url, timeout=20)
            if respuesta.ok:
                return BeautifulSoup(respuesta.text, "html.parser")
        except requests.RequestException as e:
            print(f"⚠️ Error al obtener {url}: {e}")
        time.sleep(espera * (intento + 1))
    raise Exception(f"No se pudo obtener la página después de {reintentos} intentos: {url}")

def obtener_categorias():
    sopa = obtener_sopa(URL_BASE)
    categorias = []
    for enlace in sopa.select("div.side_categories ul li ul li a"):
        nombre = enlace.get_text(strip=True)
        url = urljoin(URL_BASE, enlace.get("href"))
        categorias.append({"nombre": nombre, "url": url})
    return categorias

def recorrer_paginas_categoria(url_categoria, espera=0.8):
    url = url_categoria
    while url:
        sopa = obtener_sopa(url)
        yield url, sopa
        boton_siguiente = sopa.select_one("li.next a")
        url = urljoin(url, boton_siguiente["href"]) if boton_siguiente else None
        time.sleep(espera)

def leer_tarjetas_libros(sopa_pagina, url_pagina):
    for tarjeta in sopa_pagina.select("article.product_pod"):
        # Título del libro
        titulo = tarjeta.h3.a.get("title", "Sin título")

        # Extraer precio en texto
        precio_tag = tarjeta.find('p', class_='price_color')
        precio_str = precio_tag.text.strip() if precio_tag else "£0.00"
        precio_str = precio_str.replace("£", "").replace(",", ".").strip()

        # Convertir a número seguro
        try:
            precio = float(precio_str)
        except ValueError:
            precio = 0.0

        # Extraer rating
        clases = tarjeta.p.get("class", [])
        palabra_rating = next((c for c in clases if c != "star-rating"), "Zero")
        rating = MAPA_RATING.get(palabra_rating, 0)

        # URL de detalle del libro
        enlace_rel = tarjeta.h3.a.get("href", "")
        url_detalle = urljoin(url_pagina, enlace_rel)

        yield {
            "titulo": titulo,
            "precio": precio,
            "rating": rating,
            "url_detalle": url_detalle
        }

def leer_detalle_libro(url_detalle):
    sopa = obtener_sopa(url_detalle)
    desc_tag = sopa.select_one("#product_description ~ p")
    descripcion = desc_tag.get_text(strip=True) if desc_tag else ""
    datos = {}
    for fila in sopa.select("table.table.table-striped tr"):
        clave = fila.th.get_text(strip=True)
        valor = fila.td.get_text(strip=True)
        datos[clave] = valor
    upc = datos.get("UPC", "")
    try: precio_sin_imp = float(datos.get("Price (excl. tax)", "£0").replace("£","").replace(",", "."))
    except: precio_sin_imp = 0.0
    try: precio_con_imp = float(datos.get("Price (incl. tax)", "£0").replace("£","").replace(",", "."))
    except: precio_con_imp = 0.0
    try: impuesto = float(datos.get("Tax", "£0").replace("£","").replace(",", "."))
    except: impuesto = 0.0
    stock_texto = datos.get("Availability","")
    m = re.search(r"(\d+)", stock_texto)
    stock = int(m.group(1)) if m else 0
    palabra_rating = "Zero"
    p = sopa.select_one("p.star-rating")
    if p:
        for c in p.get("class", []):
            if c != "star-rating":
                palabra_rating = c
                break
    rating = MAPA_RATING.get(palabra_rating, 0)
    img = sopa.select_one("#product_gallery img")
    url_imagen = urljoin(url_detalle, img["src"]) if img else None
    migas = [li.get_text(strip=True) for li in sopa.select(".breadcrumb li")]
    categoria = migas[-2] if len(migas)>=2 else None
    return {
        "upc": upc,
        "precio_sin_impuesto": precio_sin_imp,
        "precio_con_impuesto": precio_con_imp,
        "impuesto": impuesto,
        "stock_texto": stock_texto,
        "stock": stock,
        "descripcion": descripcion,
        "rating": rating,
        "url_imagen": url_imagen,
        "categoria_detalle": categoria,
    }

def scrapear_todo(espera=0.3):
    resultados = []
    vistos = set()
    categorias = obtener_categorias()
    for cat in categorias:
        print(f"📚 Categoría: {cat['nombre']}")
        for url_pagina, sopa in recorrer_paginas_categoria(cat["url"], espera=espera):
            for libro in leer_tarjetas_libros(sopa, url_pagina):
                try:
                    detalle = leer_detalle_libro(libro["url_detalle"])
                except Exception as e:
                    print(f"⚠️ Error al leer detalle de {libro['titulo']}: {e}")
                    continue
                if detalle["upc"] in vistos:
                    continue
                vistos.add(detalle["upc"])
                categoria_final = detalle["categoria_detalle"] or cat["nombre"]
                resultados.append({
                    "categoria": categoria_final,
                    "titulo": libro["titulo"],
                    "precio": libro["precio"],
                    "rating": libro["rating"],
                    "upc": detalle["upc"],
                    "descripcion": detalle["descripcion"],
                    "precio_sin_impuesto": detalle["precio_sin_impuesto"],
                    "precio_con_impuesto": detalle["precio_con_impuesto"],
                    "impuesto": detalle["impuesto"],
                    "stock": detalle["stock"],
                    "url_imagen": detalle["url_imagen"],
                    "url": libro["url_detalle"],
                    "autor": f"Autor {random.randint(1,100)}"
                })
                time.sleep(espera)
    return resultados

# ------------------------------
# FUNCIONES BASE DE DATOS
# ------------------------------
def crear_bd(nombre="books.db"):
    conn = sqlite3.connect(nombre)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Autores (
        id INTEGER PRIMARY KEY,
        nombre TEXT UNIQUE
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Libros (
        id INTEGER PRIMARY KEY,
        titulo TEXT,
        precio REAL,
        rating INTEGER,
        upc TEXT UNIQUE,
        descripcion TEXT,
        precio_sin_impuesto REAL,
        precio_con_impuesto REAL,
        impuesto REAL,
        stock INTEGER,
        url_imagen TEXT,
        url TEXT,
        categoria TEXT
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Libros_Autores (
        libro_id INTEGER,
        autor_id INTEGER,
        PRIMARY KEY(libro_id, autor_id),
        FOREIGN KEY(libro_id) REFERENCES Libros(id),
        FOREIGN KEY(autor_id) REFERENCES Autores(id)
    )
    """)
    conn.commit()
    return conn

def insertar_libros_autores(conn, libros):
    cursor = conn.cursor()
    for libro in libros:
        cursor.execute("INSERT OR IGNORE INTO Autores(nombre) VALUES (?)", (libro["autor"],))
        cursor.execute("SELECT id FROM Autores WHERE nombre = ?", (libro["autor"],))
        autor_id = cursor.fetchone()[0]
        cursor.execute("""
        INSERT OR IGNORE INTO Libros(titulo, precio, rating, upc, descripcion, precio_sin_impuesto,
        precio_con_impuesto, impuesto, stock, url_imagen, url, categoria)
        VALUES (?,?,?,?,?,?,?,?,?,?,?,?)
        """, (
            libro["titulo"], libro["precio"], libro["rating"], libro["upc"], libro["descripcion"],
            libro["precio_sin_impuesto"], libro["precio_con_impuesto"], libro["impuesto"],
            libro["stock"], libro["url_imagen"], libro["url"], libro["categoria"]
        ))
        cursor.execute("SELECT id FROM Libros WHERE upc=?", (libro["upc"],))
        libro_id = cursor.fetchone()[0]
        cursor.execute("INSERT OR IGNORE INTO Libros_Autores(libro_id, autor_id) VALUES (?,?)",
                       (libro_id, autor_id))
    conn.commit()

# ------------------------------
# BLOQUE PRINCIPAL
# ------------------------------
if __name__ == "__main__":
    print("🚀 Iniciando scraping de Books to Scrape...")
    libros = scrapear_todo(espera=0.3)
    print("\n✅ Scraping finalizado.")
    print(f"Se encontraron {len(libros)} libros en total.\n")

    print("💾 Creando base de datos y guardando libros...")
    conn = crear_bd()
    insertar_libros_autores(conn, libros)
    print("✅ Datos guardados en SQLite!")

    # ------------------------------
    # CONSULTAS DE EJEMPLO
    # ------------------------------
    cursor = conn.cursor()
    print("\n📊 Ejemplo consultas:")

    print("\n1️⃣ Libros >3 estrellas y precio < £10:")
    for row in cursor.execute("SELECT titulo, precio, rating FROM Libros WHERE rating>3 AND precio<10 LIMIT 5"):
        print(row)

    print("\n2️⃣ Libros por Autor 1:")
    for row in cursor.execute("""
    SELECT l.titulo, a.nombre FROM Libros l
    JOIN Libros_Autores la ON l.id=la.libro_id
    JOIN Autores a ON a.id=la.autor_id
    WHERE a.nombre='Autor 1' LIMIT 5
    """):
        print(row)

    print("\n3️⃣ Libros de la categoría Travel:")
    for row in cursor.execute("SELECT titulo, categoria FROM Libros WHERE categoria='Travel' LIMIT 5"):
        print(row)

    print("\n4️⃣ Consulta lenta (sin índice):")
    import time
    start = time.time()
    cursor.execute("SELECT * FROM Libros WHERE precio>0 ORDER BY titulo")
    cursor.fetchall()
    print("Tiempo:", round(time.time()-start, 3), "seg")

    cursor.execute("CREATE INDEX IF NOT EXISTS idx_precio ON Libros(precio)")
    conn.commit()

    print("\n5️⃣ Consulta rápida (con índice):")
    start = time.time()
    cursor.execute("SELECT * FROM Libros WHERE precio>0 ORDER BY titulo")
    cursor.fetchall()
    print("Tiempo:", round(time.time()-start, 3), "seg")

    conn.close()
    print("\n🎉 Todo listo. Scraper + SQLite + consultas ejecutables.")



🚀 Iniciando scraping de Books to Scrape...
📚 Categoría: Travel
📚 Categoría: Mystery
📚 Categoría: Historical Fiction
📚 Categoría: Sequential Art
📚 Categoría: Classics
📚 Categoría: Philosophy
📚 Categoría: Romance
📚 Categoría: Womens Fiction
📚 Categoría: Fiction
📚 Categoría: Childrens
📚 Categoría: Religion
📚 Categoría: Nonfiction
📚 Categoría: Music
📚 Categoría: Default
📚 Categoría: Science Fiction
📚 Categoría: Sports and Games
📚 Categoría: Add a comment
📚 Categoría: Fantasy
📚 Categoría: New Adult
📚 Categoría: Young Adult
📚 Categoría: Science
📚 Categoría: Poetry
📚 Categoría: Paranormal
📚 Categoría: Art
📚 Categoría: Psychology
📚 Categoría: Autobiography
📚 Categoría: Parenting
📚 Categoría: Adult Fiction
📚 Categoría: Humor
📚 Categoría: Horror
📚 Categoría: History
📚 Categoría: Food and Drink
📚 Categoría: Christian Fiction
📚 Categoría: Business
📚 Categoría: Biography
📚 Categoría: Thriller
📚 Categoría: Contemporary
📚 Categoría: Spirituality
📚 Categoría: Academic
📚 Categoría: Self Help
📚 Categorí

In [ ]:
# ------------------------------
# SCRAPER + SQLITE COMPLETO (CORREGIDO + CONSULTAS EXTRA)
# ------------------------------

import time
import re
import sqlite3
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup
import random

# ------------------------------
# CONFIGURACIÓN SCRAPER
# ------------------------------
URL_BASE = "https://books.toscrape.com/"
CABECERAS = {"User-Agent": "Mozilla/5.0 (educational scraping)"}

MAPA_RATING = {"Zero":0, "One":1, "Two":2, "Three":3, "Four":4, "Five":5}

# Sesión requests
sesion = requests.Session()
sesion.headers.update(CABECERAS)

# ------------------------------
# FUNCIONES SCRAPER
# ------------------------------
def obtener_sopa(url, reintentos=3, espera=1.0):
    for intento in range(reintentos):
        try:
            respuesta = sesion.get(url, timeout=20)
            if respuesta.ok:
                return BeautifulSoup(respuesta.text, "html.parser")
        except requests.RequestException as e:
            print(f"⚠️ Error al obtener {url}: {e}")
        time.sleep(espera * (intento + 1))
    raise Exception(f"No se pudo obtener la página después de {reintentos} intentos: {url}")

def obtener_categorias():
    sopa = obtener_sopa(URL_BASE)
    categorias = []
    for enlace in sopa.select("div.side_categories ul li ul li a"):
        nombre = enlace.get_text(strip=True)
        url = urljoin(URL_BASE, enlace.get("href"))
        categorias.append({"nombre": nombre, "url": url})
    return categorias

def recorrer_paginas_categoria(url_categoria, espera=0.8):
    url = url_categoria
    while url:
        sopa = obtener_sopa(url)
        yield url, sopa
        boton_siguiente = sopa.select_one("li.next a")
        url = urljoin(url, boton_siguiente["href"]) if boton_siguiente else None
        time.sleep(espera)

def leer_tarjetas_libros(sopa_pagina, url_pagina):
    for tarjeta in sopa_pagina.select("article.product_pod"):
        titulo = tarjeta.h3.a.get("title", "Sin título")

        # Extraer precio en texto y limpiar caracteres
        precio_tag = tarjeta.find('p', class_='price_color')
        precio_str = precio_tag.text.strip() if precio_tag else "0.00"
        precio_str = re.sub(r"[^\d.,]", "", precio_str).replace(",", ".")
        try:
            precio = float(precio_str)
        except:
            precio = 0.0

        clases = tarjeta.p.get("class", [])
        palabra_rating = next((c for c in clases if c != "star-rating"), "Zero")
        rating = MAPA_RATING.get(palabra_rating, 0)

        enlace_rel = tarjeta.h3.a.get("href", "")
        url_detalle = urljoin(url_pagina, enlace_rel)

        yield {
            "titulo": titulo,
            "precio": precio,
            "rating": rating,
            "url_detalle": url_detalle
        }

def leer_detalle_libro(url_detalle):
    sopa = obtener_sopa(url_detalle)
    desc_tag = sopa.select_one("#product_description ~ p")
    descripcion = desc_tag.get_text(strip=True) if desc_tag else ""
    datos = {}
    for fila in sopa.select("table.table.table-striped tr"):
        clave = fila.th.get_text(strip=True)
        valor = fila.td.get_text(strip=True)
        datos[clave] = valor
    upc = datos.get("UPC", "")

    def limpiar_precio(valor):
        v = re.sub(r"[^\d.,]", "", valor).replace(",", ".")
        try:
            return float(v)
        except:
            return 0.0

    precio_sin_imp = limpiar_precio(datos.get("Price (excl. tax)", "0"))
    precio_con_imp = limpiar_precio(datos.get("Price (incl. tax)", "0"))
    impuesto = limpiar_precio(datos.get("Tax", "0"))

    stock_texto = datos.get("Availability","")
    m = re.search(r"(\d+)", stock_texto)
    stock = int(m.group(1)) if m else 0

    palabra_rating = "Zero"
    p = sopa.select_one("p.star-rating")
    if p:
        for c in p.get("class", []):
            if c != "star-rating":
                palabra_rating = c
                break
    rating = MAPA_RATING.get(palabra_rating, 0)

    img = sopa.select_one("#product_gallery img")
    url_imagen = urljoin(url_detalle, img["src"]) if img else None

    migas = [li.get_text(strip=True) for li in sopa.select(".breadcrumb li")]
    categoria = migas[-2] if len(migas)>=2 else None

    return {
        "upc": upc,
        "precio_sin_impuesto": precio_sin_imp,
        "precio_con_impuesto": precio_con_imp,
        "impuesto": impuesto,
        "stock_texto": stock_texto,
        "stock": stock,
        "descripcion": descripcion,
        "rating": rating,
        "url_imagen": url_imagen,
        "categoria_detalle": categoria
    }

def scrapear_todo(espera=0.3):
    resultados = []
    vistos = set()
    categorias = obtener_categorias()
    for cat in categorias:
        print(f"📚 Categoría: {cat['nombre']}")
        for url_pagina, sopa in recorrer_paginas_categoria(cat["url"], espera=espera):
            for libro in leer_tarjetas_libros(sopa, url_pagina):
                try:
                    detalle = leer_detalle_libro(libro["url_detalle"])
                except Exception as e:
                    print(f"⚠️ Error al leer detalle de {libro['titulo']}: {e}")
                    continue
                if detalle["upc"] in vistos:
                    continue
                vistos.add(detalle["upc"])
                categoria_final = detalle["categoria_detalle"] or cat["nombre"]
                resultados.append({
                    "categoria": categoria_final,
                    "titulo": libro["titulo"],
                    "precio": libro["precio"],
                    "rating": libro["rating"],
                    "upc": detalle["upc"],
                    "descripcion": detalle["descripcion"],
                    "precio_sin_impuesto": detalle["precio_sin_impuesto"],
                    "precio_con_impuesto": detalle["precio_con_impuesto"],
                    "impuesto": detalle["impuesto"],
                    "stock": detalle["stock"],
                    "url_imagen": detalle["url_imagen"],
                    "url": libro["url_detalle"],
                    # Autor ficticio
                    "autor": f"Autor {random.randint(1,100)}"
                })
                time.sleep(espera)
    return resultados

# ------------------------------
# FUNCIONES BASE DE DATOS
# ------------------------------
def crear_bd(nombre="books.db"):
    conn = sqlite3.connect(nombre)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Autores (
        id INTEGER PRIMARY KEY,
        nombre TEXT UNIQUE
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Libros (
        id INTEGER PRIMARY KEY,
        titulo TEXT,
        precio REAL,
        rating INTEGER,
        upc TEXT UNIQUE,
        descripcion TEXT,
        precio_sin_impuesto REAL,
        precio_con_impuesto REAL,
        impuesto REAL,
        stock INTEGER,
        url_imagen TEXT,
        url TEXT,
        categoria TEXT
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Libros_Autores (
        libro_id INTEGER,
        autor_id INTEGER,
        PRIMARY KEY(libro_id, autor_id),
        FOREIGN KEY(libro_id) REFERENCES Libros(id),
        FOREIGN KEY(autor_id) REFERENCES Autores(id)
    )
    """)
    conn.commit()
    return conn

def insertar_libros_autores(conn, libros):
    cursor = conn.cursor()
    for libro in libros:
        # Insertar o actualizar autor
        cursor.execute("INSERT OR IGNORE INTO Autores(nombre) VALUES (?)", (libro["autor"],))
        cursor.execute("SELECT id FROM Autores WHERE nombre = ?", (libro["autor"],))
        autor_id = cursor.fetchone()[0]

        # Insertar o reemplazar libro
        cursor.execute("""
        INSERT OR REPLACE INTO Libros(id, titulo, precio, rating, upc, descripcion, precio_sin_impuesto,
        precio_con_impuesto, impuesto, stock, url_imagen, url, categoria)
        VALUES (
            (SELECT id FROM Libros WHERE upc=?),
            ?,?,?,?,?,?,?,?,?,?,?,?
        )
        """, (
            libro["upc"], libro["titulo"], libro["precio"], libro["rating"], libro["upc"],
            libro["descripcion"], libro["precio_sin_impuesto"], libro["precio_con_impuesto"],
            libro["impuesto"], libro["stock"], libro["url_imagen"], libro["url"], libro["categoria"]
        ))

        # Relación libro-autor
        cursor.execute("SELECT id FROM Libros WHERE upc=?", (libro["upc"],))
        libro_id = cursor.fetchone()[0]
        cursor.execute("INSERT OR IGNORE INTO Libros_Autores(libro_id, autor_id) VALUES (?,?)",
                       (libro_id, autor_id))
    conn.commit()

# ------------------------------
# BLOQUE PRINCIPAL
# ------------------------------
if __name__ == "__main__":
    print("🚀 Iniciando scraping de Books to Scrape...")
    libros = scrapear_todo(espera=0.3)
    print("\n✅ Scraping finalizado.")
    print(f"Se encontraron {len(libros)} libros en total.\n")

    print("💾 Creando base de datos y guardando libros...")
    conn = crear_bd()
    insertar_libros_autores(conn, libros)
    print("✅ Datos guardados en SQLite!")

    # ------------------------------
    # CONSULTAS DE EJEMPLO
    # ------------------------------
    cursor = conn.cursor()

    print("\n1️⃣ Libros >3 estrellas y precio < £10:")
    for row in cursor.execute("SELECT titulo, precio, rating FROM Libros WHERE rating>3 AND precio<10 LIMIT 5"):
        print(row)

    print("\n2️⃣ Libros por Autor 1:")
    for row in cursor.execute("""
    SELECT l.titulo, a.nombre FROM Libros l
    JOIN Libros_Autores la ON l.id=la.libro_id
    JOIN Autores a ON a.id=la.autor_id
    WHERE a.nombre='Autor 1' LIMIT 5
    """):
        print(row)

    print("\n3️⃣ Libros de la categoría Travel:")
    for row in cursor.execute("SELECT titulo, categoria FROM Libros WHERE categoria='Travel' LIMIT 5"):
        print(row)

    # ------------------------------
    # DEMOSTRACIÓN DE ÍNDICE
    # ------------------------------
    print("\n🔍 Plan de ejecución SIN índice:")
    for row in cursor.execute("EXPLAIN QUERY PLAN SELECT * FROM Libros WHERE precio>0 ORDER BY titulo"):
        print(row)

    print("\n4️⃣ Consulta lenta (sin índice):")
    start = time.time()
    cursor.execute("SELECT * FROM Libros WHERE precio>0 ORDER BY titulo")
    cursor.fetchall()
    print("Tiempo:", round(time.time()-start, 3), "seg")

    cursor.execute("CREATE INDEX IF NOT EXISTS idx_precio ON Libros(precio)")
    conn.commit()

    print("\n🔍 Plan de ejecución CON índice:")
    for row in cursor.execute("EXPLAIN QUERY PLAN SELECT * FROM Libros WHERE precio>0 ORDER BY titulo"):
        print(row)

    print("\n5️⃣ Consulta rápida (con índice):")
    start = time.time()
    cursor.execute("SELECT * FROM Libros WHERE precio>0 ORDER BY titulo")
    cursor.fetchall()
    print("Tiempo:", round(time.time()-start, 3), "seg")

    # ------------------------------
    # 6 CONSULTAS EXTRA
    # ------------------------------
    print("\n📌 Todos los libros con sus autores:")
    for row in cursor.execute("""
    SELECT l.titulo, a.nombre AS autor
    FROM Libros l
    JOIN Libros_Autores la ON l.id = la.libro_id
    JOIN Autores a ON la.autor_id = a.id
    LIMIT 10
    """):
        print(row)

    print("\n📌 Contar cuántos libros tiene cada autor:")
    for row in cursor.execute("""
    SELECT a.nombre AS autor, COUNT(la.libro_id) AS total_libros
    FROM Autores a
    JOIN Libros_Autores la ON a.id = la.autor_id
    GROUP BY a.nombre
    ORDER BY total_libros DESC
    """):
        print(row)

    print("\n📌 Listar libros con precio menor a £20 y su autor:")
    for row in cursor.execute("""
    SELECT l.titulo, l.precio, a.nombre AS autor
    FROM Libros l
    JOIN Libros_Autores la ON l.id = la.libro_id
    JOIN Autores a ON la.autor_id = a.id
    WHERE l.precio < 20
    ORDER BY l.precio ASC
    """):
        print(row)

    print("\n📌 Libros con más de 3 estrellas y precio entre £10 y £30:")
    for row in cursor.execute("""
    SELECT l.titulo, l.precio, l.rating, a.nombre AS autor
    FROM Libros l
    JOIN Libros_Autores la ON l.id = la.libro_id
    JOIN Autores a ON la.autor_id = a.id
    WHERE l.rating > 3 AND l.precio BETWEEN 10 AND 30
    ORDER BY l.rating DESC, l.precio ASC
    """):
        print(row)

    print("\n📌 Cantidad de libros por categoría:")
    for row in cursor.execute("""
    SELECT l.categoria, COUNT(*) AS total_libros
    FROM Libros l
    GROUP BY l.categoria
    ORDER BY total_libros DESC
    """):
        print(row)

    print("\n📌 Libros y autores de la categoría 'Travel':")
    for row in cursor.execute("""
    SELECT l.titulo, a.nombre AS autor, l.categoria
    FROM Libros l
    JOIN Libros_Autores la ON l.id = la.libro_id
    JOIN Autores a ON la.autor_id = a.id
    WHERE l.categoria = 'Travel'
    ORDER BY l.titulo ASC
    """):
        print(row)

    conn.close()
    print("\n🎉 Todo listo. Scraper + SQLite + consultas ejecutables.")



🚀 Iniciando scraping de Books to Scrape...
📚 Categoría: Travel
📚 Categoría: Mystery
📚 Categoría: Historical Fiction
📚 Categoría: Sequential Art
📚 Categoría: Classics
📚 Categoría: Philosophy
📚 Categoría: Romance
📚 Categoría: Womens Fiction
📚 Categoría: Fiction
📚 Categoría: Childrens
📚 Categoría: Religion
📚 Categoría: Nonfiction
📚 Categoría: Music
📚 Categoría: Default
📚 Categoría: Science Fiction
📚 Categoría: Sports and Games
📚 Categoría: Add a comment
📚 Categoría: Fantasy
📚 Categoría: New Adult
📚 Categoría: Young Adult
📚 Categoría: Science
📚 Categoría: Poetry
📚 Categoría: Paranormal
📚 Categoría: Art
📚 Categoría: Psychology
📚 Categoría: Autobiography
📚 Categoría: Parenting
📚 Categoría: Adult Fiction
📚 Categoría: Humor
📚 Categoría: Horror
📚 Categoría: History
📚 Categoría: Food and Drink
📚 Categoría: Christian Fiction
📚 Categoría: Business
📚 Categoría: Biography
📚 Categoría: Thriller
📚 Categoría: Contemporary
📚 Categoría: Spirituality
📚 Categoría: Academic
📚 Categoría: Self Help
📚 Categorí

In [1]:
#Traer todos los libros con sus autores

SELECT l.titulo, a.nombre AS autor
FROM Libros l
JOIN Libros_Autores la ON l.id = la.libro_id
JOIN Autores a ON la.autor_id = a.id
LIMIT 10;

SyntaxError: invalid syntax (1119706168.py, line 3)

In [ ]:
#Contar cuántos libros tiene cada autor

SELECT a.nombre AS autor, COUNT(la.libro_id) AS total_libros
FROM Autores a
JOIN Libros_Autores la ON a.id = la.autor_id
GROUP BY a.nombre
ORDER BY total_libros DESC;

In [ ]:
#Listar libros con precio menor a £20 y su autoR
SELECT l.titulo, l.precio, a.nombre AS autor
FROM Libros l
JOIN Libros_Autores la ON l.id = la.libro_id
JOIN Autores a ON la.autor_id = a.id
WHERE l.precio < 20
ORDER BY l.precio ASC;


In [ ]:
#Libros con más de 3 estrellas y precio entre £10 y £30
SELECT l.titulo, l.precio, l.rating, a.nombre AS autor
FROM Libros l
JOIN Libros_Autores la ON l.id = la.libro_id
JOIN Autores a ON la.autor_id = a.id
WHERE l.rating > 3 AND l.precio BETWEEN 10 AND 30
ORDER BY l.rating DESC, l.precio ASC;


In [ ]:
#CANTIDAD DE LIBROS POR CATEGORIA
SELECT l.categoria, COUNT(*) AS total_libros
FROM Libros l
GROUP BY l.categoria
ORDER BY total_libros DESC;


In [ ]:
# TRAER LIBROS Y AUTORES DE UNA CATEGORIA ESPECIFICA

SELECT l.titulo, a.nombre AS autor, l.categoria
FROM Libros l
JOIN Libros_Autores la ON l.id = la.libro_id
JOIN Autores a ON la.autor_id = a.id
WHERE l.categoria = 'Travel'
ORDER BY l.titulo ASC;
